In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from os import listdir
from os.path import isfile, join, isdir
import re
from matplotlib import pyplot as plt
import seaborn as sns
import glob
import re
from scipy.interpolate import interp1d
from tqdm import tqdm, tqdm_notebook
from scipy import spatial

In [2]:
def calculate_pad(brightness, saturation):
    p = 0.69*brightness + 0.22*saturation
    a = -0.31*brightness + 0.6*saturation
    d = 0.76*brightness + 0.32*saturation
    return [p,a,d]

def calculate_pad_scene(scene):
    pads = []
    for img in scene:
        temp_b = mean_brightness(img)
        temp_s = mean_saturation(img)
        pads.append(calculate_pad(temp_b, temp_s))
    return np.mean([x[0] for x in pads]), np.mean([x[1] for x in pads]), np.mean([x[2] for x in pads])

def calculate_blur(img):
    return cv2.Laplacian(img, cv2.CV_64F).var()

def calculate_blur_scene(scene):
    blurs = []
    for img in scene:
        blurs.append(calculate_blur(img))
    return np.mean(blurs)

def mean_brightness(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    return np.mean(hsv[:,:,2])

def mean_saturation(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert it to hsv
    return np.mean(hsv[:,:,1])

def calculate_opticalFlow(img1, img2):
    prev = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    forward = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(prev)
    mask[..., 1] = 255
    flow = cv2.calcOpticalFlowFarneback(prev, forward, flow=None, pyr_scale=0.5,levels =1,winsize=3,iterations=15,poly_n=3,poly_sigma=5,flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    return cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)[0]

def calculate_opticalFlow_scene(scene):
    first = calculate_opticalFlow(scene[0], scene[1])
    second = calculate_opticalFlow(scene[1], scene[2])
    return np.mean([first, second])
    
def isjpg(filepath):
    return re.search(".jpg$", filepath)

def calculate_features(img_path):
    img = cv2.imread(img_path)
    